In [1]:
pip install tensorflow


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/375.9 MB ? eta -:--:--
                                              0.1/375.9 MB 2.8 MB/s eta 0:02:13
                                             1.4/375.9 MB 15.0 MB/s eta 0:00:25
                                             2.4/375.9 MB 16.8 MB/s eta 0:00:23
                                             2.7/375.9 MB 17.5 MB/s eta 0:00:22
                                             2.7/375.9 MB 17.5 MB/s eta 0:00:22
                                             2.7/375.9 MB 17.5 MB/s eta 0:00:22
                                             2.7/375.9 MB 17.5 MB/s eta 0:00:22
                                             2.7/375.9 MB 17.5 MB/s eta 0:00:22
                                             2.7/375.9 MB 17.5 MB/s eta 0:00:22
                                              3.5/375.9 MB 7.2 MB/s eta 0:00:52
                                              4.1/375.9 MB 7.7 MB/s eta 0:00:49
                                              

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd

# Load the dataset
train_df = pd.read_csv("Testing.csv")

# Check for any unnamed columns
train_df = train_df.loc[:, ~train_df.columns.str.contains('^Unnamed')]

# Encode the target variable
label_encoder = LabelEncoder()
train_df['prognosis'] = label_encoder.fit_transform(train_df['prognosis'])

# Normalize the features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train_df.drop(columns=['prognosis']))
y_train = train_df['prognosis'].values

# Reshape for LSTM
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

# Define the LSTM model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Save the model
model.save("disease_lstm_model.h5")


c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 819ms/step - accuracy: 0.0000e+00 - loss: 3.7150 - val_accuracy: 0.0000e+00 - val_loss: 3.7164
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.1427 - loss: 3.7096 - val_accuracy: 0.0000e+00 - val_loss: 3.7184
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.0612 - loss: 3.7054 - val_accuracy: 0.0000e+00 - val_loss: 3.7203
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.0919 - loss: 3.7036 - val_accuracy: 0.0000e+00 - val_loss: 3.7221
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.0919 - loss: 3.7012 - val_accuracy: 0.0000e+00 - val_loss: 3.7240
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.0612 - loss: 3.6987 - val_accuracy: 0.0000e+00 - val_loss: 3.7260
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.1225 - loss: 3.6935 - val_accuracy: 0.0000e+00 - val_loss: 3.7280
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.0919 - loss: 3

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Load the trained LSTM model
model = tf.keras.models.load_model("disease_lstm_model.h5")

# Load the test dataset
test_df = pd.read_csv("Testing.csv")

# Drop the 'Unnamed: 133' column if it exists
if "Unnamed: 133" in test_df.columns:
    test_df = test_df.drop(columns=["Unnamed: 133"])

# Load training dataset to ensure consistent encoding
train_df = pd.read_csv("Training.csv")
if "Unnamed: 133" in train_df.columns:
    train_df = train_df.drop(columns=["Unnamed: 133"])

# Encode the target variable (prognosis)
label_encoder = LabelEncoder()
train_df['prognosis'] = label_encoder.fit_transform(train_df['prognosis'])

# Ensure 'prognosis' is present in test data before encoding
if 'prognosis' in test_df.columns:
    y_test = label_encoder.transform(test_df['prognosis'])  # Encode test labels
    X_test = test_df.drop(columns=['prognosis'])
else:
    y_test = None  # No actual labels available
    X_test = test_df

# Normalize features using the same scaler as training
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Reshape for LSTM input
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get class index with highest probability

# Decode predictions
predicted_diseases = label_encoder.inverse_transform(y_pred_classes)

# Display the results
results = pd.DataFrame({"Predicted Disease": predicted_diseases})

# If actual labels exist, compare with predictions
if y_test is not None:
    results.insert(0, "Actual Disease", label_encoder.inverse_transform(y_test))

print(results)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step
                             Actual Disease             Predicted Disease
0                          Fungal infection                   hepatitis A
1                                   Allergy           Peptic ulcer diseae
2                                      GERD           Peptic ulcer diseae
3                       Chronic cholestasis                   hepatitis A
4                             Drug Reaction                      Jaundice
5                       Peptic ulcer diseae           Peptic ulcer diseae
6                                      AIDS  Dimorphic hemmorhoids(piles)
7                                 Diabetes                      Diabetes 
8                           Gastroenteritis           Peptic ulcer diseae
9                          Bronchial Asthma                   hepatitis A
10                            Hypertension   Dimorphic hemmorhoids(piles)
11                                 Migraine                      Migraine

In [1]:
pip install streamlit


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
